<a href="https://colab.research.google.com/github/leonardolunamoreno/Computadoras-y-programaci-n2021-grupo-1157/blob/main/LINK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pygame, sys # Con esta línea se importan las librerías de pygame 
from settings import * # Con está línea se importan los datos de la siguiente ventana " settings " que es donde aparece el tamaño de la ventana y algunos otros detalles como las posiciones de los jugadores, enemigos u otros obejtos gráficos
from level import Level # Aquí se importa la pestaña del nivel
# from debug import debug # Se importa debug, con la finalidad de mostrar como hace las pruebas internas del juego (ir a línea 21 )

class Game:  # Aquí se define la clase juego: Game
    def __init__(self): # Aquí def__init__(self) es un método 

        # Configuración general del juego 
        pygame.init()
        self.screen = pygame.display.set_mode((WIDTH,HEIGTH))  # Aquí se crea una superficie de exhibición, que se importa desde settings
        pygame.display.set_caption('Leonardo Link') # Aquí le cambiamos el nombre a la ventana que muestra nuestro videojuego, el nombre aparecerá en la esquina superior izquierda
        self.clock = pygame.time.Clock() # Aquí se crea un reloj 
       
        self.level = Level() # Aquí se crea una instancia de la pestaña videojuego 

    def run(self):       # Aquí se define parte de la carrera del método
        while True:      
            for event in pygame.event.get():   # Aquí se define un evento de tipo bucle
                if event.type == pygame.QUIT:  # Aquí se comprueba si se cerró el juego
                    pygame.quit()
                    sys.exit()

            self.screen.fill('darkred') # Define el color del fondo de la superficie de exibición, o la pantalla :v
            self.level.run() # Se ejecuta la instancia de la clase nivel dentro del bucle del juego, de la ventana level, que definimos previamente
            # debug('Buenos días Leonardo Alexis Luna Moreno') # Dentro de este bucle del juego se puede ejecutar la depuración, si debug está importado, pero no se escribe la sentencia en el lugar índicado, el programa sencillamente marca debug con un foco y con la información, "remove unsuport import"
            pygame.display.update()     # Actualiza la pantalla
            self.clock.tick(FPS)        # Controla el marco y tasa de cuadros por segundo, previamente importados

if __name__ == '__main__':  # Aquí se crea un método main según parece
    game = Game() # Aquí se crea una instancia de la clase juego: Game 
    game.run()    # Aquí se llama al metodo run de esa clase, la clase juego: Game        


In [ ]:
import pygame 
from settings import *
from tile import Tile
from player import Player
from Enemigo1 import Enemigo 
from debug import debug

class Level:  # Se crea la clase nivel
    def __init__(self): # Se define el método __init__() el cual tiene a self como argumento 

        # Get the display surface
        self.display_surface = pygame.display.get_surface() # Está linea sirve para obtener una pantalla 

        # Configuración de grupo de sprites, aquí se agrupan los dos grandes grupos de sprites dentro del juego 
        self.visible_sprites = YSortCameraGroup()    
        self.obstacles_sprites = pygame.sprite.Group()

        # Configuración de los sprites
        self.create_map() # Aquí se crea parte del mapa

    def create_map(self): 
        for row_index,row in enumerate(WORLD_MAP): # utlizamos está línea para obtener algunos detalles sobre las posiciones del jugador en el mapa
            for col_index,col in enumerate(row):
                x = col_index * TILESIZE    # print(row_index) utilizamos estas lineas para obtener información del mapa 
                y = row_index * TILESIZE    # print(row) lo mismo que arriba
                if col == 'x': # Aquí se coloca un mosaico desde la pestaña mosaico, el cual también aparece en settings
                    Tile((x,y),[self.visible_sprites,self.obstacles_sprites]) # Para lograr lo se necesitan 2 argumentos, la posición y los grupos
               
                if col == 'e':
                    Enemigo((x,y),[self.visible_sprites,self.obstacles_sprites])

                if col == 'p': # Aquí se coloca al jugador
                    self.player = Player((x,y),[self.visible_sprites],self.obstacles_sprites) # Aquí se agrega la posición de jugador y el argumento sprite visible

    def run(self):
         # Actualizar y dibujar el juego
        self.visible_sprites.custom_draw(self.player) # Con esta línea es posible mostrar los sprites que hayamos definido previamente
        self.visible_sprites.update()     # pass     Está linea fue utilizada antes de agregar la línea con el argumento " self.visible_sprites.draw(self.display_surface) "
        # debug(self.player.direction) # Al agregar estas lineas y correr el programa, aparece en la esquina superior iquierda un recuadro donde se indica si las teclas de dirección son pulsadas o no

class YSortCameraGroup(pygame.sprite.Group):
    def __init__(self):

        #Configuracion general 
        super().__init__()        
        self.display_surface = pygame.display.get_surface()
        self.half_width = self.display_surface.get_size()[0] // 2
        self.half_height = self.display_surface.get_size()[1] // 2
        self.offset = pygame.math.Vector2()
    
    def custom_draw(self,player):
        # Obtener el Offset 
        self.offset.x = player.rect.centerx - self.half_width
        self.offset.y = player.rect.centery - self.half_height
        #for sprite in self.sprites():
        for sprite in sorted(self.sprites(), key = lambda sprite: sprite.rect.centery):
            offset_pos = sprite.rect.topleft - self.offset 
            self.display_surface.blit(sprite.image,offset_pos)

In [ ]:
import pygame  
from settings import *  

class Player(pygame.sprite.Sprite):
    def __init__(self,pos,groups,obstacle_sprites):
       super().__init__(groups)
       self.image = pygame.image.load('../graphics/test/leonardo0.png').convert_alpha()  # Aquí se agrega la imagen que se utilizará para el sprite del jugador 
       self.rect = self.image.get_rect(topleft = pos)
       self.hitbox = self.rect.inflate(0,-20)


       self.direction = pygame.math.Vector2() # De esta manera obtenemos un vector cuyos valores son 0,0, y a su vez podemos cambiar la entrada de estos dos números con la entrada de alguna dirección desde el teclado
       self.speed = 4

       self.obstacle_sprites = obstacle_sprites 


    def input(self):
        keys = pygame.key.get_pressed() # Con esté método se obtiene la entrada de los mandos desde el teclado

        if keys[pygame.K_DOWN]:
            self.direction.y = 1 
        elif keys[pygame.K_UP]:
            self.direction.y = -1 
        else: 
            self.direction.y = 0 

        if keys[pygame.K_LEFT]:
            self.direction.x = -1
        elif keys[pygame.K_RIGHT]:
            self.direction.x = 1
        else: 
            self.direction.x = 0

    def move(self,speed): # Aquí se crea el método moverse, con los argumentos self y velocidad 
        if self.direction.magnitude() != 0:  # Aquí nos cercioramos que nuestro vector tiene cuaquier tipo de longitud
            self.direction = self.direction.normalize()  # Está línea cambia la longitud del vector, sea cual sea, al valor de 1 
      
        self.hitbox.x += self.direction.x * speed
        self.collision('horizontal')
        self.hitbox.y += self.direction.y * speed 
        self.collision('vertical')
        self.rect.center = self.hitbox.center
        #self.rect.center += self.direction * speed # Aquí se multiplica la tecla de entrada por una determinada velocidad a la cual se puee mover el personaje 
            
    def collision(self,direction):
        if direction == 'horizontal':
            for sprite in self.obstacle_sprites: 
                if sprite.hitbox.colliderect(self.hitbox):
                    if self.direction.x > 0: # Moverse a la derecha 
                        self.hitbox.right = sprite.hitbox.left # Moverse a la derecha
                    if self.direction.x < 0:     # Moverse a la izquierda
                        self.hitbox.left = sprite.hitbox.right  #Moverse a la izquierda 
        
        if direction == 'vertical':
            for sprite in self.obstacle_sprites: 
                if sprite.hitbox.colliderect(self.hitbox): # Aquí aunque colliderect no esta marcado de colores el juego todavia compila 
                    if self.direction.y > 0: # Moverse hacia abajo                    
                        self.hitbox.bottom = sprite.hitbox.top
                    if self.direction.y < 0:
                        self.hitbox.top = sprite.hitbox.bottom
    


    def update(self):   # Aquí se agrega otro método más
        self.input()
        self.move(self.speed)
       

In [ ]:
 # Configuración general del juego
WIDTH = 1280 # Aquí se determina el ancho de la ventana
HEIGTH = 720 # Aquí se determina la altura de la ventana 
FPS = 60      # Cantidad de cuadros mostrados por segundo 
TILESIZE = 64 # Tamaño del mosaico, no se exactamente que signifique 

WORLD_MAP = [
['x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x'],
['x',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
['x',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'], # Dentro del mapa mundial, la letra p representa la posición del jugador en el espacio, además, se nota de lejos que el mapa es un arreglo de matrices 
['x',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
['x',' ',' ','x',' ','e','e','e','e','x','x','x','x',' ',' ',' ',' ',' ',' ','x'], # Posición del jugador
['x',' ',' ','x',' ','e',' ',' ','e','x',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'], # x = 2 * 64 = 128 
['x',' ',' ','x',' ','e',' ','p','e','x',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'], # y = 2 * 64 = 128 
['x',' ',' ','x',' ','e',' ',' ','e','x',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
['x',' ',' ','x',' ','e',' ',' ','e','x',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
['x',' ',' ','x',' ',' ',' ',' ',' ','x',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
['x',' ',' ','x',' ',' ',' ',' ',' ','x',' ',' ',' ','x',' ',' ',' ',' ',' ','x'],
['x',' ',' ','x',' ',' ',' ',' ',' ','x',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
['x',' ',' ','x',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','x','x',' ',' ',' ','x'],
['x',' ',' ','x',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
['x',' ',' ','x',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
['x',' ',' ','x','x','x',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
['x',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
['x',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
['x',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
['x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x'],
]


In [ ]:
import pygame  
from settings import *  # Tile significa mosaico 

class Enemigo(pygame.sprite.Sprite):
    def __init__(self,pos,groups):
        super().__init__(groups)
        self.image = pygame.image.load('../graphics/test/Crow1.png').convert_alpha() # El tamaño de la imagen que simule las rocas debería de ser de 64 x 64 esto con la finalidad de que no queden espacios en el mapa de bits 
        self.rect = self.image.get_rect(topleft = pos)
        self.hitbox = self.rect.inflate(0,-10)

In [ ]:
import pygame  
from settings import *  # Importación de configuraciones

class Tile(pygame.sprite.Sprite): # Tile significa mosaico, Acá dentro del parentesis se define la herencia de los sprites
    def __init__(self,pos,groups):
        super().__init__(groups)
        self.image = pygame.image.load('../graphics/test/ojodefinitivo.png').convert_alpha() # El tamaño de la imagen que simule las rocas debería de ser de 64 x 64 esto con la finalidad de que no queden espacios en el mapa de bits 
        self.rect = self.image.get_rect(topleft = pos)
        self.hitbox = self.rect.inflate(0,-30)
        

In [ ]:
import pygame  # Este es el archivo de depuración
pygame.init()  # No influye directamente en el juego, pero es de gran utilidad al momento de realizar pruebas 
font = pygame.font.Font(None,30)  # La depuración sirve para demostar que todo funciona correctamente dentro del juego
                                  # o aplicación que estemos creando en determinado momento
def debug(info, y = 10, x = 10):
    display_surface = pygame.display.get_surface()
    debug_surf = font.render(str(info),True,'White')
    debug_rect = debug_surf.get_rect(topleft = (x,y))
    pygame.draw.rect(display_surface, 'Black', debug_rect)
    display_surface.blit(debug_surf,debug_rect)


In [ ]:
# Después de mucho tiempo, logré recuperar el videojuego, al menos hasta un punto anterior con el código aquí escrito el programa aun funciona. 
# Hoy es la madrugada del 28 de Diciembre del 2022